In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
import os
import argparse
import numpy as np
import json

import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image
import networkx as nx
from matplotlib import pyplot as plt

from minimal.arch import Generator

from minimal.dataset import (
    FloorplanGraphDataset,
    floorplan_collate_fn,
    reader
)

from minimal.utils import (
    init_input,
    draw_masks,
    draw_graph
)

%matplotlib inline

In [3]:
DATA_PATH = "./data/sample_list.txt"

In [4]:
data_path = DATA_PATH
transform = transforms.Normalize(mean=[0.5], std=[0.5])
target_set = 8
split = "test"

In [5]:
subgraphs = []

In [6]:
with open(data_path, "r") as f1:
    lines = f1.readlines()
    lines = list(map(lambda x: x.strip(), lines))

lines

['data/json/45012.json',
 'data/json/45161.json',
 'data/json/36233.json',
 'data/json/19307.json',
 'data/json/7513.json',
 'data/json/18477.json']

In [7]:
# for line in lines:
#     rms_type, fp_eds, rms_bbs, eds_to_rms, eds_to_rms_tmp = reader(line)
    
#     subgraphs.append([
#         rms_type,
#         rms_bbs,
#         fp_eds,
#         eds_to_rms,
#         eds_to_rms_tmp,
#     ])

In [33]:
# rms_type, fp_eds, rms_bbs, eds_to_rms, eds_to_rms_tmp = reader("data/expr.json")

# rms_type: [5, 3, 3, 4, 2, 1, 17, 17, 17, 17, 17, 15]
# fp_eds: (54,4)
# rms_bbs: (12, 4): same shape as "boxes" from file, but contents
#                   modified/adjusted a little
# eds_to_rms: list[list[int]] "ed_rm" from file
# eds_to_rms_tmp: eds_to_rms but the inner list truncated to length 1
#                 i.e eds_to_rms_tmp[i][0] == eds_to_rms[i][0] for every i

# print(eds_to_rms_tmp)

In [27]:
with open("data/json/45012.json") as f:
    info = json.load(f)

In [41]:
# read from file
rms_type = info["room_type"]
rms_bbs = np.array(info["boxes"])
fp_eds = np.array(info["edges"])
eds_to_rms = info["ed_rm"]

print(rms_bbs.shape)
print(fp_eds.shape)

(12, 4)
(54, 6)


In [39]:
# Count rooms that are not equal to 17 (interior_door)
s_r = 0
for rmk in range(len(rms_type)):
    if rms_type[rmk] != 17:
        s_r = s_r + 1

print(f"{s_r = }")

s_r = 7


In [43]:
# convert bounding boxes from range [0,256] to range [0,1]
# rms_bbs = rms_bbs / 256.0
# fp_eds = fp_eds / 256.0

In [45]:
# discard last 2 number of fp_eds
# new shape: (*, 4)
fp_eds = fp_eds[:, :4]
print(fp_eds.shape)

(54, 4)


In [57]:
# (R, 2)
rooms_top_lefts = rms_bbs[:, :2]
rooms_btm_rights = rms_bbs[:, 2:]

# top left of the bounding box of the masks
global_tl = np.min(rooms_top_lefts, axis=0)
# bottom right of the bounding box of the masks
global_br = np.max(rooms_btm_rights, axis=0)

In [62]:
# Make the center of the overall bounding box (0.5, 0.5)
# i.e centralize the floor plan 
shift = (global_tl + global_br) / 2.0 - 0.5
# rms_bbs[:, :2] -= shift
# rms_bbs[:, 2:] -= shift
# fp_eds[:, :2] -= shift
# fp_eds[:, 2:] -= shift